#Estratégia de Coleta e Inconsistências Históricas
##1. Arquitetura do Script de Coleta (Crawler)
Para lidar com o volume massivo de dados (25 anos de histórico da aviação civil), este script foi desenvolvido utilizando estratégias de Big Data e Resiliência:

Streaming & Chunking: Os dados não são carregados integralmente na memória RAM. O download é realizado mês a mês e os dados são imediatamente escritos no disco (Google Drive) em modo append (anexar), evitando estouro de memória.

Checkpointing: O sistema salva um arquivo de texto auxiliar (checkpoint.txt) contendo a última data processada. Em caso de falha de conexão ou timeout do Colab, o script lê este arquivo e retoma exatamente de onde parou, evitando duplicidade de dados.

Tratamento de JSON: Foi implementada uma correção automática para casos onde a API da ANAC retorna objetos JSON encapsulados indevidamente como strings, o que causava erros de parsing.

##2. O Desafio da Integridade de Dados (Pós-2017)
Durante a validação do arquivo histórico completo (aprox. 4.6 GB), identificou-se um fenômeno conhecido como "Schema Drift" (Mudança de Estrutura):

O Problema: Ferramentas tradicionais (Excel e Pandas) conseguiam ler apenas os dados até Dezembro de 2017, ignorando os anos seguintes, embora o tamanho do arquivo indicasse que os dados estavam lá.

A Causa: A partir de janeiro de 2018, a ANAC alterou a nomenclatura das colunas (ex: mudou de PartidaReal para dt_partida_real) e o padrão de formatação. Como o CSV foi gerado com o cabeçalho do ano 2000, os leitores estruturados descartaram os dados novos por não corresponderem às colunas originais.

A Solução: Para garantir a qualidade do modelo preditivo, optou-se por realizar uma extração focada e normalizada apenas do período de interesse para o MVP (2023 e 2024), garantindo que todas as colunas de data e status estejam padronizadas e limpas.

## Importações
### CONFIGURAÇÃO DE AMBIENTE
### Se estiver rodando localmente, altere os caminhos na seção: CONFIGURAÇÃO E MONTAGEM DO DRIVE,  para o seu diretório local.

In [34]:
import requests
import pandas as pd
import urllib3
from datetime import date, timedelta, datetime
import time
import os
import json
from google.colab import drive

In [35]:
data_inicio_geral = date(2023, 1, 1)
data_fim_geral = date(2023, 3, 31)

##  CONFIGURAÇÃO E MONTAGEM DO DRIVE

##Leitura dos dados

Nesta seção deve ser disponibilizado data para realizar a coleta dos dados.

In [36]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

print("Montando Google Drive...")
drive.mount('/content/drive')

# Defina a pasta no seu Drive onde os dados ficarão
pasta_drive = "/content/drive/My Drive/Dados_ANAC"
os.makedirs(pasta_drive, exist_ok=True) # Cria a pasta se não existir

# Nomes dos arquivos (Caminho completo)
periodo_nome = f"{data_inicio_geral.strftime('%b%Y')}_a_{data_fim_geral.strftime('%b%Y')}"
arquivo_saida = os.path.join(pasta_drive, f"dados_anac_{periodo_nome}.csv")

arquivo_checkpoint = os.path.join(pasta_drive, "checkpoint_anac.txt")

url_base = "https://sas.anac.gov.br/sas/vra_api/vra"

Montando Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Definição da função que divide o período total em intervalos mensais.

In [37]:
# Função geradora de datas
def datas_por_mes(inicio, fim):
    atual = inicio
    while atual <= fim:
        inicio_mes = atual.replace(day=1)
        proximo_mes = (inicio_mes.replace(day=28) + timedelta(days=4)).replace(day=1)
        fim_mes = proximo_mes - timedelta(days=1)
        if fim_mes > fim: fim_mes = fim
        yield inicio_mes, fim_mes
        atual = proximo_mes

##LÓGICA DE RETOMADA
Implementa a lógica de checkpoint e retomada: verifica se há um arquivo de progresso salvo para continuar o download do mês seguinte ao último processado, evitando recomeçar do zero.
**Deletar o arquivo chekpoint para novas leituras, ou não executar esta parte.**

In [38]:
# Verifica se já existe um checkpoint para continuar
if os.path.exists(arquivo_checkpoint):

    with open(arquivo_checkpoint, 'r') as f:
        data_salva = f.read().strip()

        try:
            # Converte texto volta para data e avança para o próximo mês
            data_parada = datetime.strptime(data_salva, "%Y-%m-%d").date()

            # Define o início como o primeiro dia do mês seguinte
            proximo_mes = (data_parada.replace(day=28) + timedelta(days=4)).replace(day=1)
            data_inicio_geral = proximo_mes
            print(f"Checkpoint encontrado! Retomando download a partir de: {data_inicio_geral}")

        except:
            print("Erro ao ler checkpoint. Começando do zero.")

else:
    print("Iniciando download do zero.")

# Define se precisamos escrever o cabeçalho (apenas se o arquivo não existir)
escrever_cabecalho = not os.path.exists(arquivo_saida)
total_registros = 0
start_time = time.time()

print(f" Salvando dados em: {arquivo_saida}")

Iniciando download do zero.
 Salvando dados em: /content/drive/My Drive/Dados_ANAC/dados_anac_Jan2023_a_Mar2023.csv


##LOOP PRINCIPAl (STREAMING)
Loop principal que itera sobre os intervalos mensais, realiza as requisições à API da ANAC com sistema de 3 tentativas (retry) para falhas de rede, trata erros de formatação no JSON recebido e salva os dados incrementalmente no arquivo CSV (append), atualizando o checkpoint a cada mês concluído com sucesso.

In [39]:

# ##LOOP PRINCIPAl (STREAMING)
for dt_ini, dt_fim in datas_por_mes(data_inicio_geral, data_fim_geral):

    p1 = dt_ini.strftime("%d%m%Y")
    p2 = dt_fim.strftime("%d%m%Y")

    params = {"dt_referencia1": p1, "dt_referencia2": p2}

    sucesso_no_mes = False
    tentativas = 0

    # Pequeno loop de retry para instabilidade de rede
    while not sucesso_no_mes and tentativas < 3:
        try:
            response = requests.get(url_base, params=params, verify=False, timeout=120)

            if response.status_code == 200:
                dados_brutos = response.json()

                # Tratamento do JSON dentro de String (Bug da API)
                if isinstance(dados_brutos, str):
                    try:
                        dados_processados = json.loads(dados_brutos)
                    except:
                        print(f"  [ERRO PARSE JSON] {p1}-{p2}")
                        dados_processados = []
                else:
                    dados_processados = dados_brutos

                if isinstance(dados_processados, list) and len(dados_processados) > 0:
                    df_temp = pd.DataFrame(dados_processados)

                    # Salva no Drive (Append Mode)
                    df_temp.to_csv(arquivo_saida, mode='a', header=escrever_cabecalho, index=False, encoding='utf-8', sep=';')

                    qtd = len(df_temp)
                    total_registros += qtd

                    # Atualiza flags
                    escrever_cabecalho = False
                    sucesso_no_mes = True

                    # --- SALVA O CHECKPOINT ---
                    with open(arquivo_checkpoint, 'w') as f:
                        f.write(dt_ini.strftime("%Y-%m-%d"))

                    print(f"{dt_ini.strftime('%b/%Y')}: +{qtd} voos salvos.")

                    del df_temp
                    del dados_processados
                else:
                    print(f"{dt_ini.strftime('%b/%Y')}: Mês vazio.")
                    sucesso_no_mes = True # Considera sucesso para avançar
                    # Salva checkpoint mesmo se vazio para não travar
                    with open(arquivo_checkpoint, 'w') as f:
                        f.write(dt_ini.strftime("%Y-%m-%d"))

            else:
                print(f"Erro API {response.status_code}. Tentativa {tentativas+1}/3")
                tentativas += 1
                time.sleep(2)

        except Exception as e:
            print(f"Erro Conexão: {e}. Tentativa {tentativas+1}/3")
            tentativas += 1
            time.sleep(5)

    # Pausa de cortesia para a API
    time.sleep(0.5)

print("\n" + "="*40)
print("Processo finalizado ou pausado.")
print(f"Arquivo CSV atualizado em: {arquivo_saida}")

Jan/2023: +87647 voos salvos.
Feb/2023: +73701 voos salvos.
Mar/2023: +82268 voos salvos.

Processo finalizado ou pausado.
Arquivo CSV atualizado em: /content/drive/My Drive/Dados_ANAC/dados_anac_Jan2023_a_Mar2023.csv


##Visualização do csv gerado

In [43]:
print(f"Lendo dados de :{arquivo_saida}")

!head -n 10 "{arquivo_saida}"
print("\n" + "="*40 + "\n")
!tail -n 10 "{arquivo_saida}"

Lendo dados de :/content/drive/My Drive/Dados_ANAC/dados_anac_Jan2023_a_Mar2023.csv
sg_empresa_icao;nm_empresa;nr_voo;cd_di;cd_tipo_linha;sg_equipamento_icao;nr_assentos_ofertados;sg_icao_origem;nm_aerodromo_origem;dt_partida_prevista;dt_partida_real;sg_icao_destino;nm_aerodromo_destino;dt_chegada_prevista;dt_chegada_real;ds_situacao_voo;ds_justificativa;dt_referencia;ds_situacao_partida;ds_situacao_chegada
AAL;AMERICAN AIRLINES, INC.;0904;0;I;B788;295;SBGL;AEROPORTO INTERNACIONAL DO RIO DE JANEIRO (GALEÃO) - ANTONIO CARLOS JOBIM - RIO DE JANEIRO - RJ - BRASIL;01/01/2023 00:05;31/12/2022 23:59;KMIA;MIAMI INTERNATIONAL AIRPORT - MIAMI, FLORIDA - ESTADOS UNIDOS DA AMÉRICA;01/01/2023 08:37;01/01/2023 08:12;REALIZADO;;01/01/2023;Antecipado;Antecipado
AAL;AMERICAN AIRLINES, INC.;0905;0;I;B788;295;KMIA;MIAMI INTERNATIONAL AIRPORT - MIAMI, FLORIDA - ESTADOS UNIDOS DA AMÉRICA;01/01/2023 21:45;01/01/2023 00:56;SBGL;AEROPORTO INTERNACIONAL DO RIO DE JANEIRO (GALEÃO) - ANTONIO CARLOS JOBIM - RIO 